# Tutorial 1: Features & Visibility

**Goal:** Understand HERON's core design and create FeatureProviders that define **what** agents observe and **who** can see it.

**Time:** ~15 minutes

---

## Why HERON?

Training RL agents for multi-agent systems (power grids, traffic, robotics) requires solving:

| Challenge | Traditional Approach | HERON Solution |
|-----------|---------------------|----------------|
| Partial observability | Manual filtering code per agent | Declarative `visibility = ['owner', 'upper_level']` |
| Hierarchical control | Hardcoded parent-child logic | Built-in `FieldAgent` → `CoordinatorAgent` → `SystemAgent` |
| Realistic timing | Single execution mode | Dual modes: sync (training) + event-driven (testing) |
| Swappable coordination | Rewrite code per protocol | Pluggable `SetpointProtocol`, `PriceSignalProtocol` |

### The Key Insight: Agent-Centric Design

**Traditional (environment-centric):** Environment manages everything, agents are stateless policy wrappers.

**HERON (agent-centric):** Agents are first-class citizens with state, timing, and hierarchy. Features declare their own visibility.

```python
# Traditional: 50+ lines of manual filtering
def get_obs_for_battery(global_state, agent_id):
    if agent_id == 'battery_1': return global_state['battery_1']['soc']
    elif agent_id == 'microgrid_1': return global_state['battery_1']['soc']
    # ... more filtering logic

# HERON: 2 lines - visibility is declarative
class BatterySOC(FeatureProvider):
    visibility = ['owner', 'upper_level']  # Done. No filtering code needed.
```

---

## The Key Idea: Declarative Visibility

With HERON, visibility is **declared**, not coded:

```python
class BatterySOC(FeatureProvider):
    visibility = ['owner', 'upper_level']  # Battery + coordinator can see
    soc: float = 0.5
```

### Visibility Levels

| Level | Who Can See | Example Use |
|-------|------------|-------------|
| `public` | Everyone | Time of day, market prices |
| `owner` | The owning agent | Internal device state |
| `upper_level` | Owner + parent in hierarchy | Battery SOC visible to microgrid |
| `system` | System operator (top level) | Grid-wide metrics |

## The FeatureProvider Interface

Every feature implements these methods:

```python
@dataclass
class MyFeature(FeatureProvider):
    visibility = ['owner', 'upper_level']  # Who can see this
    
    value: float = 0.0  # Your data
    
    def vector(self) -> np.ndarray:       # → ML observation
        return np.array([self.value])
    
    def names(self) -> List[str]:         # → Debug/logging
        return ['value']
    
    def to_dict(self) -> Dict:            # → Serialization
        return {'value': self.value}
    
    @classmethod
    def from_dict(cls, d) -> 'MyFeature': # → Deserialization
        return cls(**d)
    
    def set_values(self, **kwargs):       # → Update values
        if 'value' in kwargs:
            self.value = kwargs['value']
```

That's the full interface. Now let's build some real features.

## Example 1: Battery State of Charge

**Visibility:** Owner (battery) + upper_level (microgrid coordinator)

In [ ]:
from dataclasses import dataclass
from typing import Any, Dict, List
import numpy as np

from heron.core.feature import FeatureProvider


@dataclass
class BatterySOC(FeatureProvider):
    """Battery state of charge. Visible to owner and coordinator."""
    visibility = ['owner', 'upper_level']
    
    soc: float = 0.5  # State of charge [0, 1]
    
    def vector(self) -> np.ndarray:
        return np.array([self.soc], dtype=np.float32)
    
    def names(self) -> List[str]:
        return ['soc']
    
    def to_dict(self) -> Dict:
        return {'soc': self.soc}
    
    @classmethod
    def from_dict(cls, d) -> 'BatterySOC':
        return cls(**d)
    
    def set_values(self, **kw):
        if 'soc' in kw:
            self.soc = np.clip(float(kw['soc']), 0.0, 1.0)


# Test it
battery_soc = BatterySOC(soc=0.75)
print(f"Vector: {battery_soc.vector()}")
print(f"Visibility: {battery_soc.visibility}")

## Example 2: Generator Output

**Visibility:** Owner + upper_level + system (grid operator needs to see total generation)

In [ ]:
@dataclass
class GenOutput(FeatureProvider):
    """Generator power output. Visible to owner, coordinator, AND system."""
    visibility = ['owner', 'upper_level', 'system']
    
    p_mw: float = 0.0   # Power output [MW]
    p_max: float = 5.0  # Maximum capacity [MW]
    
    def vector(self) -> np.ndarray:
        # Normalize to [0, 1] for better ML training
        p_norm = self.p_mw / max(self.p_max, 1e-6)
        return np.array([p_norm], dtype=np.float32)
    
    def names(self) -> List[str]:
        return ['p_normalized']
    
    def to_dict(self) -> Dict:
        return {'p_mw': self.p_mw, 'p_max': self.p_max}
    
    @classmethod
    def from_dict(cls, d) -> 'GenOutput':
        return cls(**d)
    
    def set_values(self, **kw):
        if 'p_mw' in kw:
            self.p_mw = float(kw['p_mw'])


# Test it
gen = GenOutput(p_mw=2.5, p_max=5.0)
print(f"Vector (normalized): {gen.vector()}")  # [0.5]
print(f"Raw p_mw: {gen.p_mw} MW")

## Example 3: Public Information

**Visibility:** `public` — everyone sees this (time of day, market prices)

In [ ]:
@dataclass
class TimeOfDay(FeatureProvider):
    """Time feature - visible to everyone."""
    visibility = ['public']
    
    hour: int = 0  # 0-23
    
    def vector(self) -> np.ndarray:
        # Encode cyclically (hour 23 is close to hour 0)
        rad = 2 * np.pi * self.hour / 24
        return np.array([np.sin(rad), np.cos(rad)], dtype=np.float32)
    
    def names(self): return ['hour_sin', 'hour_cos']
    def to_dict(self): return {'hour': self.hour}
    @classmethod
    def from_dict(cls, d): return cls(hour=d['hour'])
    def set_values(self, **kw):
        if 'hour' in kw: self.hour = int(kw['hour']) % 24


# Test
time = TimeOfDay(hour=14)  # 2 PM
print(f"Time vector: {time.vector()}")
print(f"Visibility: {time.visibility}")

## How Visibility Filtering Works

HERON automatically checks visibility based on agent hierarchy:

In [ ]:
# BatterySOC has visibility = ['owner', 'upper_level']
battery_soc = BatterySOC(soc=0.8)

# Check who can see this feature:

# 1. Owner (the battery itself) - YES
can_owner_see = battery_soc.is_observable_by(
    requestor_id='battery_1', requestor_level=1,
    owner_id='battery_1', owner_level=1
)
print(f"Battery can see its own SOC: {can_owner_see}")  # True

# 2. Coordinator (upper_level) - YES  
can_coord_see = battery_soc.is_observable_by(
    requestor_id='microgrid_1', requestor_level=2,  # One level above
    owner_id='battery_1', owner_level=1
)
print(f"Coordinator can see battery SOC: {can_coord_see}")  # True

# 3. Another battery (peer, not upper_level) - NO
can_peer_see = battery_soc.is_observable_by(
    requestor_id='battery_2', requestor_level=1,  # Same level, not owner
    owner_id='battery_1', owner_level=1
)
print(f"Other battery can see SOC: {can_peer_see}")  # False

## State Classes for All Hierarchy Levels

HERON provides three state classes matching the 3-level agent hierarchy:

| Level | Agent Type | State Class | Typical Features |
|-------|------------|-------------|------------------|
| L1 | FieldAgent | `FieldAgentState` | Device state (SOC, power output) |
| L2 | CoordinatorAgent | `CoordinatorAgentState` | Aggregated metrics, local load |
| L3 | SystemAgent | `SystemAgentState` | System-wide metrics (frequency, total generation) |

All three work the same way: create the state, append features, use `vector()` or `observed_by()`.

In [ ]:
from heron.core.state import FieldAgentState, CoordinatorAgentState, SystemAgentState

# ===== L1: FieldAgentState (device level) =====
# Create state for a battery agent
battery_state = FieldAgentState(owner_id='battery_1', owner_level=1)

# Add feature to state (use features.append, not register_feature)
battery_state.features.append(BatterySOC(soc=0.7))

# Get full observation vector (all features)
full_obs = battery_state.vector()
print(f"Full state vector: {full_obs}")

# Get observation filtered by requestor
# (In practice, HERON does this automatically based on agent hierarchy)
filtered_obs = battery_state.observed_by(requestor_id='battery_1', requestor_level=1)
print(f"Filtered observation: {filtered_obs}")


# ===== L2: CoordinatorAgentState (coordinator level) =====
# Define a coordinator-level feature (aggregated metrics)
@dataclass
class AggregatedLoad(FeatureProvider):
    """Total load across all subordinate devices."""
    visibility = ['owner', 'system']  # Coordinator + system operator
    
    total_mw: float = 0.0
    
    def vector(self) -> np.ndarray:
        return np.array([self.total_mw / 100.0], dtype=np.float32)  # Normalize
    
    def names(self): return ['total_load_normalized']
    def to_dict(self): return {'total_mw': self.total_mw}
    @classmethod
    def from_dict(cls, d): return cls(**d)
    def set_values(self, **kw):
        if 'total_mw' in kw: self.total_mw = float(kw['total_mw'])

coord_state = CoordinatorAgentState(owner_id='microgrid_1', owner_level=2)
coord_state.features.append(AggregatedLoad(total_mw=45.0))
print(f"CoordinatorAgentState vector: {coord_state.vector()}")

# ===== L3: SystemAgentState (system level) =====
# Define a system-level feature (grid-wide metrics)
@dataclass
class SystemFrequency(FeatureProvider):
    """Grid frequency - system-wide metric."""
    visibility = ['system']  # Only system operator sees this
    
    frequency_hz: float = 60.0
    nominal_hz: float = 60.0
    
    def vector(self) -> np.ndarray:
        deviation = (self.frequency_hz - self.nominal_hz) / self.nominal_hz
        return np.array([deviation], dtype=np.float32)
    
    def names(self): return ['frequency_deviation']
    def to_dict(self): return {'frequency_hz': self.frequency_hz}
    @classmethod
    def from_dict(cls, d): return cls(**d)
    def set_values(self, **kw):
        if 'frequency_hz' in kw: self.frequency_hz = float(kw['frequency_hz'])

system_state = SystemAgentState(owner_id='grid_system', owner_level=3)
system_state.features.append(SystemFrequency(frequency_hz=59.95))
print(f"SystemAgentState vector: {system_state.vector()}")

# All three state types use the same API pattern!

## Design Guidelines

| Feature Type | Visibility | Example |
|--------------|------------|---------|
| Private device state | `['owner']` | Internal temperature |
| Hierarchical info | `['owner', 'upper_level']` | Battery SOC, generator output |
| System metrics | `['owner', 'upper_level', 'system']` | Total generation |
| Public info | `['public']` | Time, weather, prices |

**Tips:**
- **Normalize in `vector()`**: ML algorithms prefer [0, 1] or [-1, 1]
- **Store raw values**: Keep originals for debugging
- **Use `@dataclass`**: Clean definitions, auto-generated `__init__`

## Summary: Features We Built

```python
# L1 (Field) Features
BatterySOC      →  visibility = ['owner', 'upper_level']
GenOutput       →  visibility = ['owner', 'upper_level', 'system']
TimeOfDay       →  visibility = ['public']

# L2 (Coordinator) Features
AggregatedLoad  →  visibility = ['owner', 'system']

# L3 (System) Features
SystemFrequency →  visibility = ['system']
```

These features will be used by agents in the next tutorial.

In [ ]:
# Verify our features work
print("Features defined successfully!")
print(f"  BatterySOC: {BatterySOC(soc=0.5).vector()}")
print(f"  GenOutput: {GenOutput(p_mw=2.5, p_max=5.0).vector()}")
print(f"  TimeOfDay: {TimeOfDay(hour=12).vector()}")

## Key Takeaways

1. **HERON is agent-centric** — Agents have state, timing, hierarchy (not just policy wrappers)
2. **Visibility is declarative** — Just set `visibility = ['owner', 'upper_level']`
3. **No manual filtering** — HERON handles observation filtering automatically
4. **Three state classes for three hierarchy levels**:
   - `FieldAgentState` (L1) — Device-level state
   - `CoordinatorAgentState` (L2) — Coordinator-level aggregated metrics
   - `SystemAgentState` (L3) — System-wide metrics
5. **Normalize for ML** — Return [0,1] in `vector()`, keep raw values internally

---

**Next:** [02_building_agents.ipynb](02_building_agents.ipynb) — Create agents that use these features